Get local copy of a tsv table.

In [45]:
!cp ../../data/exercise-3/GLOBAL/COI-trim30min100-merge-c3-id97-OTU-denovo.tsv .

Get metadata.

In [46]:
!cp ../../data/exercise-5/metadata.csv .

Parse BIOM tsv table and transpose for importing into R.

In [47]:
def BIOM_tsv_to_R_transpose(in_tsv, out_csv):
    """
    Parse a biom table in tsv format and transpose it for input into R
    """
    
    from biom import Table
    
    tsv = open(in_tsv)
    #in_tsv = open('COI-trim30min100-merge-c3-id97-OTU-taxonomy.kraken.tsv')
    func = lambda x : x
    intable = Table.from_tsv(tsv,obs_mapping=None, sample_mapping=None, process_func=func)
    outtable = intable.transpose()
    out=open("transposed.tsv","w")
    out.write(outtable.to_tsv(header_key=None, header_value=None))
    out.close()

    #refine
    intable = open('transposed.tsv','r')
    temp = intable.next()

    out=''
    for line in intable:
        if line.startswith('#'):
            if line.strip().endswith('taxomomy'):
                print "Removing taxonomy"
                line = ",".join(line.strip().split("\t")[:-1]).replace('#OTU ID','Sample').replace('\t',',')+'\n'
            line = line.replace('#OTU ID','Sample').replace('\t',',')
            out+=line
        else:
            line = line.replace('\t',',')
            out+=line

    outtable = open(out_csv,'w')
    outtable.write(out)
    outtable.close()

In [42]:
BIOM_tsv_to_R_transpose(in_tsv='COI-trim30min100-merge-c3-id97-OTU-denovo.tsv', out_csv='good_for_R.csv')


In [49]:
def adjust_metadata_sample_ids(intable, metadata):
    """
    Addjust metadata sample ids to match the sample ids in table
    """
    import sys
    
    #get sample ids in table
    ids = []
    table = open(intable,'r')
    table.next()
    
    for line in table:
        ids.append(line.split(",")[0])
        
    table.close()
#    print ids
    
    #parse metadata
    meta_in = open(metadata,'r')
    out = meta_in.next()

    meta_ids = {}
    meta_data = {}
    for line in meta_in:
        l = line.strip().split(",")
#        print l[0]
        for i in range(len(ids)):
            if ids[i].startswith(l[0]):
                if not l[0] in meta_ids:
                    meta_ids[l[0]] = []
                meta_ids[l[0]].append(ids[i])
                
                if len(meta_ids[l[0]]) == 1:
#                    print "Found matches for %s: %s" %(l[0],meta_ids[l[0]] )
                    l[0] = meta_ids[l[0]][0]
                    meta_data[i] = l
                else:
                    sys.exit("ambiguous match for %s" %(l[0]))
        else:
            "Did not find a match for %s" %l[0]
    
    meta_in.close()
    
    if len(meta_ids) == len(ids):
        to_correct = 1 #always sort 
        for s in meta_ids:
            if not s == meta_ids[s][0]:
                print "Sample ids in '%s' match those in '%s'" %(metadata, intable)
                to_correct+=1
#            else:
#                print "%s is identical to %s" %(s,meta_ids[s][0])

        if to_correct:
            print "ok - updating file %s" %metadata
            #convert
            outfh = open(metadata,'w')
            for s in sorted(meta_data):
                out+=",".join(meta_data[s])+'\n'
        
            outfh.write(out)
            outfh.close()
        
        else:
            print "Nothing to do here - sample ids in '%s' match those in '%s'" %(metadata, intable)
        
        
    else:
        print "Did not find metadata for all samples. "
#        print meta_ids
        


In [50]:
adjust_metadata_sample_ids(intable='good_for_R.csv', metadata='metadata.csv')

ok - updating file metadata.csv


Plot stuff in R.